In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Date
from datetime import datetime, timedelta

# initialising the parent class for a table
Base = declarative_base()

# a task table
class Task(Base):
    __tablename__ = 'task'
    id = Column(Integer, primary_key=True)
    task = Column(String, default='default_value')
    deadline = Column(Date, default =datetime.today())

    def __repr__(self):
        return self.task
    
    
    
class To_Do_List:  
    prompt = '\n1) Today\'s tasks\n2) Week\'s tasks\n3) All tasks\n4) Missed tasks\n5) Add task\n6) Delete task\n0) Exit\n'
    
    def __init__(self): # todo要不要引号
        """
        initialising the database and gui
        """
        # table and database initialising
        self.engine = create_engine('sqlite:///todo.db?check_same_thread=False')
        Base.metadata.create_all(self.engine)

        # session and interaction with database initialising
        self.session = sessionmaker(bind = self.engine)()
        
        # todo list gui initialising
        self.choices = {'1': self.today_tasks, '2': self.weekly_tasks, '3': self.all_tasks, '4': self.missed_tasks, '5': self.add_task, '6': self.delete_task, '0': self.shutdown}
        self.running = True 
        self.main()
    
    def shutdown(self):
        self.running = False
        
    def today_tasks(self):
        today = datetime.today()
        rows = self.session.query(Task).filter(Task.deadline == today.date()).all()
        print('Today {}:'.format(today.strftime('%b %d')))
        if rows:
            for i in range(len(rows)):
                print("{}. {}".format(i+1, rows[i].task, ))
        else:
            print('Nothing to do!')
        self.main()
    
    def weekly_tasks(self):
        rows = self.session.query(Task).order_by(Task.deadline).all()
        today = datetime.today()
        for i in range(7):
            weekday = datetime.date(today + timedelta(days=i))
            weekday_f = weekday.strftime('%A %d %b')
            print('\n' + weekday_f +':')
            flag = 0
            for i in range(len(rows)):
                
                if rows[i].deadline==weekday:
                    flag=1
                    print('{}. {}'.format(i+1, rows[i].task))
            if flag == 0:
                print('Nothing to do!\n')
        self.main()

        
    def all_tasks(self):
        rows = self.session.query(Task).order_by(Task.deadline).all()
        print('All tasks:')
        if rows:
            for row in rows:
                print("{}. {}. {}".format(row.id, row.task, row.deadline.strftime('%d %b')))
        else:
            print('Nothing to do!')
        self.main()
        
    def missed_tasks(self):
        rows = self.session.query(Task).order_by(Task.deadline).all()
        today = datetime.today().date()
        print('Missed tasks:')
        flag = 0
        for i in range(len(rows)):
            if rows[i].deadline < today:
                print('{}. {}. {}'.format(i+1, rows[i].task, rows[i].deadline.strftime('%d %b')))
                flag = 1
        if flag == 0:
            print('Nothing is missed!')
        self.main()
                
    
    def add_task(self):
        task = input('Enter task\n')
        deadline = datetime.strptime(input('Enter deadline\n'),'%Y-%m-%d').date()
        self.session.add(Task(task=task, deadline=deadline))
        self.session.commit()
        print('The task has been added!')
        
    def delete_task(self):
        print('Choose the number of the task you want to delete:')
        rows = self.session.query(Task).order_by(Task.deadline).all()
        for i in range(len(rows)):
            print('{}. {}. {}'.format(i+1, rows[i].task, rows[i].deadline.strftime('%d %b')))
        item_num = int(input())-1
        self.session.delete(rows[item_num])
        self.session.commit()

    def main(self):
        while self.running:
            choice = input(self.prompt)
            print()
            self.choices.get(choice, lambda: None)()
            print() if choice != '0' else print('Bye!')

To_Do_List()


1) Today's tasks
2) Week's tasks
3) All tasks
4) Missed tasks
5) Add task
6) Delete task
0) Exit
3

All tasks:
4. tasknew. 11 Dec
3. Deadline is today. 15 Dec
1. Deadline in 3 days. 18 Dec
2. Deadline in 6 days. 21 Dec

1) Today's tasks
2) Week's tasks
3) All tasks
4) Missed tasks
5) Add task
6) Delete task
0) Exit
4

Missed tasks:
1. tasknew. 11 Dec

1) Today's tasks
2) Week's tasks
3) All tasks
4) Missed tasks
5) Add task
6) Delete task
0) Exit
6

Choose the number of the task you want to delete:
1. tasknew. 11 Dec
2. Deadline is today. 15 Dec
3. Deadline in 3 days. 18 Dec
4. Deadline in 6 days. 21 Dec
1


1) Today's tasks
2) Week's tasks
3) All tasks
4) Missed tasks
5) Add task
6) Delete task
0) Exit
3

All tasks:
3. Deadline is today. 15 Dec
1. Deadline in 3 days. 18 Dec
2. Deadline in 6 days. 21 Dec
